


## Building the image

Clone the course repo if you haven't:

```
git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git
```

Go to the `course-zoomcamp/cohorts/2024/05-deployment/homework` folder and
execute the following:


```bash
docker build -t zoomcamp-model:3.11.5-hw10 .
```

> **Note:** If you have troubles building the image, you can
> use the image we built and published to docker hub:
> `docker pull svizor/zoomcamp-model:3.11.5-hw10`


## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:3.11.5-hw10
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```python
{'has_subscribed': True, 'has_subscribed_probability': <value>}
```

Here `<value>` is the probability of getting a subscription. You need to choose the right one.

* 0.757





Now you can stop the container running in Docker.


## Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/

## Question 2

What's the version of `kind` that you have?

Use `kind --version` to find out.

**0.25.0**


## Creating a cluster

---



Now let's create a cluster with `kind`:

```bash
kind create cluster
```

And check with `kubectl` that it was successfully created:

```bash
kubectl cluster-info
```

## Question 3

What's the smallest deployable computing unit that we can create and manage
in Kubernetes (`kind` in our case)?

**Pod**

## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?


**ClusterIP**

```bash
kubectl get services
```

```bash
NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   3m53s
```

##Question 5

To be able to use the docker image we previously created (zoomcamp-model:3.11.5-hw10), we need to register it with kind.

What's the command we need to run for that?

**kind load docker-image**

```bash
kind load docker-image zoomcamp-model:3.11.5-hw10
```

## Question 6

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: zoomcamp-model:3.11.5-hw10
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: "128Mi"
            cpu: "200m"
        ports:
        - containerPort: 9696
```

```bash
kubectl apply -f deployment.yaml
```
**Correct &ltport&gt; :9696**

```bash
kubectl get deployments
```
```
NAME           READY   UP-TO-DATE   AVAILABLE   AGE
subscription   1/1     1            1           12s
```

```bash
kubectl get pods
```

```
NAME                            READY   STATUS    RESTARTS   AGE
subscription-544b4f9664-c5z5s   1/1     Running   0          25s
```

##Question 7

Let's create a service for this deployment (service.yaml):

```
apiVersion: v1
kind: Service
metadata:
  name: subscription-service
spec:
  type: LoadBalancer
  selector:
    app: subscription
  ports:
  - port: 80
    targetPort: 9696

```
**Correct &lt;???&gt; : subscription**

##Testing the service


```
kubectl apply -f service.yaml
```

```
kubectl get services
```

```
NAME                   TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
kubernetes             ClusterIP      10.96.0.1      <none>        443/TCP        37m
subscription-service   LoadBalancer   10.96.25.159   <pending>     80:30117/TCP   11s
```

Can NOT be terminated
```
kubectl port-forward service/subscription-service 9696:80
```

```
Forwarding from 127.0.0.1:9696 -> 9696
Forwarding from [::1]:9696 -> 9696
Handling connection for 9696
```

```
python q6_test.py
{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}
```


## Autoscaling

Use the following command to create the HPA:

```bash
kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3
```

You can check the current status of the new HPA by running:
```
kubectl get hpa
```

```
kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
```

Let's see how the autoscaler reacts to increasing the load. To do this, we can slightly modify the existing `q6_test.py` script by putting the operator that sends the request to the subscription service into a loop.

```
while True:
    sleep(0.1)
    response = requests.post(url, json=client).json()
    print(response)
```

## Question 8 (optional)
Run `kubectl get hpa subscription-hpa --watch` command to monitor how the autoscaler performs. Within a minute or so, you should see the higher CPU load; and then - more replicas. What was the maximum amount of the replicas during this test?

**1**